# Importanto Bibliotecas

In [1]:
import os
import cv2
import numpy as np
import joblib
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, classification_report

# Corel 1k

## Definindo os diretórios

In [2]:
data_dir = f"./data"

#corel-1k
dataset_dir = f"{data_dir}/dataset"
images_dir = f"{dataset_dir}/images"

## Em escala de cinza:

### Gerando histogramas

In [ ]:
def obter_vizinhos(matriz_de_intensidade, linha, coluna):
    # O objetivo dessa função é retornar uma lista contendo a intensidade dos 9 vizinhos do pixel observado

    #Esse método de partição da matriz foi escolhido para manter a analise dos pixels vizinhos
    #dentro dos limites da matriz de intensidade, evitando assim valores negativos ou fora do shape.
    vizinhos = matriz_de_intensidade[max(0, linha-1):min(matriz_de_intensidade.shape[0], linha+2),
                             max(0, coluna-1):min(matriz_de_intensidade.shape[1], coluna+2)]

    #Transforma a matriz particionada em lista e remove o pixel do centro
    lista_de_vizinhos = vizinhos.flatten().tolist()
    lista_de_vizinhos.remove(matriz_de_intensidade[linha, coluna])

    return lista_de_vizinhos

In [ ]:
def aplica_regras(lista_de_vizinhos, intensidade_pixel_central, estado_pixel_central):
    #O objetivo dessa função é aplicar as regras do jogo da vida de Conway no pixel observado e retornar seu estado.
    # 1 = vivo, 0 = morto

    #Conta o número de vizinhos vivos com a mesma intensidade que o pixel central
    vizinhos_iguais = lista_de_vizinhos.count(intensidade_pixel_central)

    #Regra 1: A célula viva com dois ou três vizinhos vivos sobrevive
    if estado_pixel_central == 1 and (vizinhos_iguais == 2 or vizinhos_iguais == 3):
        return 1

    #Regra 2: A célula viva com menos de dois vizinhos vivos morre (subpopulação)
    elif estado_pixel_central == 1 and vizinhos_iguais < 2:
        return 0

    #Regra 3: A célula viva com mais de três vizinhos vivos morre (superpopulação)
    elif estado_pixel_central == 1 and vizinhos_iguais > 3:
        return 0

    #Regra 4: A célula morta com exatamente três vizinhos vivos se torna viva (resurreição)
    elif estado_pixel_central == 0 and vizinhos_iguais == 3:
        return 1

    #Pra todos os outros casos, a célula permanece no mesmo estado
    else:
        return estado_pixel_central

In [ ]:
def percorre_imagem_aplicando_regras(matriz_de_estados, matriz_de_intensidade):
    # O objetivo dessa função é percorrer a imagem, chamar a função para obter os vizinhos e aplicar as regras

    linhas, colunas = matriz_de_intensidade.shape
    for linha in range(linhas):
        for coluna in range(colunas):
            #Obtem os vizinhos do pixel atual
            lista_de_vizinhos = obter_vizinhos(matriz_de_intensidade, linha, coluna)
            #Aplica as regras do jogo da vida no pixel atual (atualiza a matriz de estado inicial)
            matriz_de_estados[linha, coluna] = aplica_regras(lista_de_vizinhos, matriz_de_intensidade[linha, coluna], matriz_de_estados[linha, coluna])

    return matriz_de_estados

In [ ]:
def gera_histogramas(imagem_cinza):
    #O objetivo dessa função é criar as matriz de intensidade e as de estado inicial para cada imagem
    #Após aplicar as regras cria os histogramas

    #Transforma a imagem em uma matriz de intensidade
    matriz_de_intensidade = np.array(imagem_cinza)

    #Cria as matrizes de estados iniciais
    matriz_de_estados_phi = np.ones(matriz_de_intensidade.shape, dtype=int) #todos vivos
    matriz_de_estados_psi = np.zeros(matriz_de_intensidade.shape, dtype=int) #todos mortos

    #Aplica as regras do jogo da vida e atualiza as matrizes de estado inicial
    matriz_de_estados_phi = percorre_imagem_aplicando_regras(matriz_de_estados_phi, matriz_de_intensidade)
    matriz_de_estados_psi = percorre_imagem_aplicando_regras(matriz_de_estados_psi, matriz_de_intensidade)

    #As matrizes são convertidas em listas
    #Phi -> estado inicial = vivo
    phi_vivos = matriz_de_intensidade.flatten()[matriz_de_estados_phi.flatten() == 1] #se manteram vivos
    phi_mortos = matriz_de_intensidade.flatten()[matriz_de_estados_phi.flatten() == 0] #morreram

    #Psi -> estado inicial = morto
    psi_vivos = matriz_de_intensidade.flatten()[matriz_de_estados_psi.flatten() == 1] #ressuscitaram
    psi_mortos = matriz_de_intensidade.flatten()[matriz_de_estados_psi.flatten() == 0] #se manteram mortos

    #Cria os histogramas
    hist_phi_vivos, _ = np.histogram(phi_vivos, bins=256, range=(0, 256))
    hist_phi_mortos, _ = np.histogram(phi_mortos, bins=256, range=(0, 256))
    hist_psi_vivos, _ = np.histogram(psi_vivos, bins=256, range=(0, 256))
    hist_psi_mortos, _ = np.histogram(psi_mortos, bins=256, range=(0, 256))

    return hist_phi_vivos, hist_phi_mortos, hist_psi_vivos, hist_psi_mortos


In [ ]:
def gerador_histogramas(dataset_dir):
    #Funcao principal: faz a chamada das funções acima e salva os histogramas

    images_dir = f"{dataset_dir}/images"
    histograms_dir = f"{dataset_dir}/histograms_grayscale"

    # Cria as pastas para salvar os histogramas mantendo o padrão das classes do dataset
    os.makedirs(histograms_dir, exist_ok=True)

    classes = [conteudo_item for conteudo_item in os.listdir(images_dir) if os.path.isdir(os.path.join(images_dir, conteudo_item))]
    for classe in classes:
        dir_pastas = os.path.join(histograms_dir, classe)
        os.makedirs(dir_pastas, exist_ok=True)

    i = 0
    #Esse loop pega a imagem, gera seus histogramas e salva com base no nome da classe e da imagem
    for classe in os.listdir(images_dir):
        dir_classe = f"{images_dir}/{classe}"

        for imagem in os.listdir(dir_classe):
            imagem_path = f"{images_dir}/{classe}/{imagem}"

            imagem_cinza = cv2.imread(imagem_path, cv2.IMREAD_GRAYSCALE)

            imagem_cinza = cv2.resize(imagem_cinza, (128,128))
            #Dimensao original = 384 * 256 = 98304
            #Resize = 128 * 128 = 16384
            #Redução = (16384-98304)/98304*100 = 83,33%

            hist_phi_vivos, hist_phi_mortos, hist_psi_vivos, hist_psi_mortos = gera_histogramas(imagem_cinza)

            index = imagem.split(".")[0]
            file_path = os.path.join(f"{histograms_dir}/{classe}", f"{index}_phi_vivos.pkl")
            joblib.dump(hist_phi_vivos, file_path)
            file_path = os.path.join(f"{histograms_dir}/{classe}", f"{index}_phi_mortos.pkl")
            joblib.dump(hist_phi_mortos, file_path)
            file_path = os.path.join(f"{histograms_dir}/{classe}", f"{index}_psi_vivos.pkl")
            joblib.dump(hist_psi_vivos, file_path)
            file_path = os.path.join(f"{histograms_dir}/{classe}", f"{index}_psi_mortos.pkl")
            joblib.dump(hist_psi_mortos, file_path)

            i += 1
            if i % 10 == 0:
              print("Progresso = %0.1f por cento" % (i / 1000 * 100))

    print("Progresso concluído")

In [ ]:
#gerador_histogramas(dataset_dir)

### Testando modelos de aprendizado de máquina combinando histogramas

#### Carregando histogramas

In [ ]:
def carrega_hist(imagem_path):
    #O objetivo dessa função é carregar os histogramas de determinada imagem

    #data\dataset\images\beaches\100.jpg
    imagem_path = imagem_path.split("/")
    classe, imagem = imagem_path[4], imagem_path[5]
    imagem = imagem.split(".")[0]

    
    histogramas_dir = f"{dataset_dir}/histograms_grayscale"
    try:
        hist_phi_vivos = joblib.load(f"{histogramas_dir}/{classe}/{imagem}_phi_vivos.pkl")
        hist_phi_mortos = joblib.load(f"{histogramas_dir}/{classe}/{imagem}_phi_mortos.pkl")
        hist_psi_vivos = joblib.load(f"{histogramas_dir}/{classe}/{imagem}_psi_vivos.pkl")
        hist_psi_mortos = joblib.load(f"{histogramas_dir}/{classe}/{imagem}_psi_mortos.pkl")

        return hist_phi_vivos, hist_phi_mortos, hist_psi_vivos, hist_psi_mortos

    except:
        print("Os histogramas não foram encontrados. Verifique os diretórios e se os histogramas foram criados corretamente.")

In [ ]:
def retorna_combinacao(imagem_path, combinacao):
    # O objetivo dessa funcao é criar combinacoes com os histogramas da imagem

    hist_phi_vivos, hist_phi_mortos, hist_psi_vivos, hist_psi_mortos = carrega_hist(imagem_path)
    if combinacao == 0:
        return hist_phi_vivos
    elif combinacao == 1:
        return hist_phi_mortos
    elif combinacao == 2:
        return hist_psi_vivos
    elif combinacao == 3:
        return hist_psi_mortos
    elif combinacao == 4:
        return np.concatenate([hist_phi_vivos, hist_phi_mortos]) #histogramas phi
    elif combinacao == 5:
        return np.concatenate([hist_psi_vivos, hist_psi_mortos]) #histogramas psi
    elif combinacao == 6:
        return np.concatenate([hist_phi_vivos, hist_psi_vivos]) #histogramas vivos
    elif combinacao == 7:
        return np.concatenate([hist_phi_mortos, hist_psi_mortos]) #histogramas mortos
    elif combinacao == 8:
        return np.concatenate([np.concatenate([hist_phi_vivos, hist_phi_mortos]),
                               np.concatenate([hist_psi_vivos, hist_psi_mortos])])
        #histogramas phi e psi combinados

In [ ]:
'''0 -> "hist_phi_vivos", 1 -> "hist_phi_mortos", 2 -> "hist_psi_vivos", 3 -> "hist_psi_mortos",
4 -> "hist_phi_combined", 5 -> "hist_psi_combined", 6 -> "hist_vivos_combined",
7 -> "hist_mortos_combined", 8 -> "all_combined" '''

#### Preparando dados

In [ ]:
# Prepara os dados para treinar o modelo

histogramas = []
rotulos = []

for classe in os.listdir(images_dir):
    dir_classe = f"{images_dir}/{classe}"
    for imagem in os.listdir(dir_classe):
        imagem_path = f"{images_dir}/{classe}/{imagem}"

        #Pode alterar a combinacao
        combinacao = retorna_combinacao(imagem_path, 8)

        #Lista dos histogramas
        histogramas.append(combinacao)
        #lista da classe assossiada ao histograma
        rotulos.append(classe)

print("Progresso concluído")

In [ ]:
X = np.array(histogramas)
y = np.array(rotulos)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1,stratify=y)

In [ ]:
#Utilizando tecnicas de normalizacao
# 1 = MinMaxScaler, 2 = StandardScaler, 3 = MaxAbsScaler, 4 = RobustScaler
selectedNormalization = 1

if selectedNormalization == 1:
    scaler = preprocessing.MinMaxScaler()
if selectedNormalization == 2:
    scaler = preprocessing.StandardScaler()
if selectedNormalization == 3:
    scaler = preprocessing.MaxAbsScaler()
if selectedNormalization == 4:
    scaler = preprocessing.RobustScaler()

# Escalando os dados de treinamento
X_train = scaler.fit_transform(X_train)
# Escalando os dados de teste com os dados de treinamento, visto que os dados de teste podem ser apenas 1 amostra
X_test = scaler.transform(X_test)

#### Testando modelos

In [ ]:
# Gaussian Naive Bayes
gnb = GaussianNB()
model1 = gnb.fit(X_train, y_train)
aux = gnb.predict(X_test)
print(classification_report(y_test, aux))
print(confusion_matrix(y_test, aux))

In [ ]:
# Logistic Regression
logreg = LogisticRegression()
model2 = logreg.fit(X_train, y_train)
aux = logreg.predict(X_test)
print(classification_report(y_test, aux))
print(confusion_matrix(y_test, aux))

In [ ]:
# Decision Tree
dectree = DecisionTreeClassifier()
model3 = dectree.fit(X_train, y_train)
aux = dectree.predict(X_test)
print(classification_report(y_test, aux))
print(confusion_matrix(y_test, aux))

In [ ]:
# K-Nearest Neighbors
knn = KNeighborsClassifier(n_neighbors = 3)
model4 = knn.fit(X_train, y_train)
aux = knn.predict(X_test)
print(classification_report(y_test, aux))
print(confusion_matrix(y_test, aux))

In [ ]:
# Linear Discriminant Analysis
lda = LinearDiscriminantAnalysis()
model5 = lda.fit(X_train, y_train)
aux = lda.predict(X_test)
print(classification_report(y_test, aux))
print(confusion_matrix(y_test, aux))

In [ ]:
# Support Vector Machine
svm = SVC()
model6 = svm.fit(X_train, y_train)
aux = svm.predict(X_test)
print(classification_report(y_test, aux))
print(confusion_matrix(y_test, aux))

In [ ]:
# RandomForest
rf = RandomForestClassifier()
model7 = rf.fit(X_train, y_train)
aux = rf.predict(X_test)
print(classification_report(y_test, aux))
print(confusion_matrix(y_test, aux))

In [ ]:
# Neural Net
nnet = MLPClassifier(alpha=1)
model8 = nnet.fit(X_train, y_train)
aux = nnet.predict(X_test)
print(classification_report(y_test, aux))
print(confusion_matrix(y_test, aux))

## Em RGB:

### Gerando histogramas

In [3]:
def obter_vizinhos_rgb(matriz_de_intensidade, linha, coluna):
    # O objetivo dessa função é retornar uma lista contendo a intensidade dos 9 vizinhos do pixel observado

    #Esse método de partição da matriz foi escolhido para manter a analise dos pixels vizinhos
    #dentro dos limites da matriz de intensidade, evitando assim valores negativos ou fora do shape.
    vizinhos = matriz_de_intensidade[max(0, linha-1):min(matriz_de_intensidade.shape[0], linha+2),
                             max(0, coluna-1):min(matriz_de_intensidade.shape[1], coluna+2)]

    #Transforma a matriz particionada em lista e remove o pixel do centro
    lista_de_vizinhos = vizinhos.flatten().tolist()
    lista_de_vizinhos.remove(matriz_de_intensidade[linha, coluna])

    return lista_de_vizinhos

In [4]:
def aplica_regras_rgb(lista_de_vizinhos, intensidade_pixel_central, estado_pixel_central):
    #O objetivo dessa função é aplicar as regras do jogo da vida de Conway no pixel observado e retornar seu estado.
    # 1 = vivo, 0 = morto

    #Conta o número de vizinhos vivos com a mesma intensidade que o pixel central
    vizinhos_iguais = lista_de_vizinhos.count(intensidade_pixel_central)

    #Regra 1: A célula viva com dois ou três vizinhos vivos sobrevive
    if estado_pixel_central == 1 and (vizinhos_iguais == 2 or vizinhos_iguais == 3):
        return 1

    #Regra 2: A célula viva com menos de dois vizinhos vivos morre (subpopulação)
    elif estado_pixel_central == 1 and vizinhos_iguais < 2:
        return 0

    #Regra 3: A célula viva com mais de três vizinhos vivos morre (superpopulação)
    elif estado_pixel_central == 1 and vizinhos_iguais > 3:
        return 0

    #Regra 4: A célula morta com exatamente três vizinhos vivos se torna viva (resurreição)
    elif estado_pixel_central == 0 and vizinhos_iguais == 3:
        return 1

    #Pra todos os outros casos, a célula permanece no mesmo estado
    else:
        return estado_pixel_central

In [5]:
def percorre_imagem_aplicando_regras_rgb(matriz_de_estados, matriz_de_intensidade):
    # O objetivo dessa função é percorrer a imagem, chamar a função para obter os vizinhos e aplicar as regras

    linhas, colunas = matriz_de_intensidade.shape
    for linha in range(linhas):
        for coluna in range(colunas):
            #Obtem os vizinhos do pixel atual
            lista_de_vizinhos = obter_vizinhos_rgb(matriz_de_intensidade, linha, coluna)
            #Aplica as regras do jogo da vida no pixel atual (atualiza a matriz de estado inicial)
            matriz_de_estados[linha, coluna] = aplica_regras_rgb(lista_de_vizinhos, matriz_de_intensidade[linha, coluna], matriz_de_estados[linha, coluna])

    return matriz_de_estados

In [6]:
def gera_histogramas_rgb(imagem):
    #O objetivo dessa função é criar as matriz de intensidade e as de estado inicial para cada imagem
    #Após aplicar as regras cria os histogramas

    #Transforma a imagem em uma matriz de intensidade
    matriz_de_intensidade = np.array(imagem)

    #Cria as matrizes de estados iniciais
    matriz_de_estados_phi = np.ones(matriz_de_intensidade.shape, dtype=int) #todos vivos
    matriz_de_estados_psi = np.zeros(matriz_de_intensidade.shape, dtype=int) #todos mortos

    #Aplica as regras do jogo da vida e atualiza as matrizes de estado inicial
    matriz_de_estados_phi = percorre_imagem_aplicando_regras_rgb(matriz_de_estados_phi, matriz_de_intensidade)
    matriz_de_estados_psi = percorre_imagem_aplicando_regras_rgb(matriz_de_estados_psi, matriz_de_intensidade)

    #As matrizes são convertidas em listas
    #Phi -> estado inicial = vivo
    phi_vivos = matriz_de_intensidade.flatten()[matriz_de_estados_phi.flatten() == 1] #se manteram vivos
    phi_mortos = matriz_de_intensidade.flatten()[matriz_de_estados_phi.flatten() == 0] #morreram

    #Psi -> estado inicial = morto
    psi_vivos = matriz_de_intensidade.flatten()[matriz_de_estados_psi.flatten() == 1] #ressuscitaram
    psi_mortos = matriz_de_intensidade.flatten()[matriz_de_estados_psi.flatten() == 0] #se manteram mortos

    #Cria os histogramas
    hist_phi_vivos, _ = np.histogram(phi_vivos, bins=256, range=(0, 256))
    hist_phi_mortos, _ = np.histogram(phi_mortos, bins=256, range=(0, 256))
    hist_psi_vivos, _ = np.histogram(psi_vivos, bins=256, range=(0, 256))
    hist_psi_mortos, _ = np.histogram(psi_mortos, bins=256, range=(0, 256))

    return hist_phi_vivos, hist_phi_mortos, hist_psi_vivos, hist_psi_mortos

In [7]:
def gerador_histogramas_RGB(dataset_dir):
    images_dir = f"{dataset_dir}/images"
    histograms_dir = f"{dataset_dir}/histograms_rgb"
    os.makedirs(histograms_dir, exist_ok=True)

    classes = [d for d in os.listdir(images_dir) if os.path.isdir(os.path.join(images_dir, d))]
    for classe in classes:
        os.makedirs(os.path.join(histograms_dir, classe), exist_ok=True)

    for classe in classes:
        print(classe)
        dir_classe = os.path.join(images_dir, classe)
        for imagem in os.listdir(dir_classe):
            try:
                imagem_path = os.path.join(dir_classe, imagem)
                imagem_rgb = cv2.imread(imagem_path, cv2.IMREAD_COLOR)
                imagem_rgb = cv2.resize(imagem_rgb, (128, 128))  # Resize para 128x128
                
                hist_phi_vivos_b, hist_phi_mortos_b, hist_psi_vivos_b, hist_psi_mortos_b = gera_histogramas_rgb(imagem_rgb[:, :, 0])
                hist_phi_vivos_g, hist_phi_mortos_g, hist_psi_vivos_g, hist_psi_mortos_g = gera_histogramas_rgb(imagem_rgb[:, :, 1])
                hist_phi_vivos_r, hist_phi_mortos_r, hist_psi_vivos_r, hist_psi_mortos_r = gera_histogramas_rgb(imagem_rgb[:, :, 2])
                
                hist_phi_vivos = np.concatenate([np.concatenate([hist_phi_vivos_b,hist_phi_vivos_g]),hist_phi_vivos_r])
                hist_phi_mortos = np.concatenate([np.concatenate([hist_phi_mortos_b,hist_phi_mortos_g]),hist_phi_mortos_r])
                hist_psi_vivos = np.concatenate([np.concatenate([hist_psi_vivos_b,hist_psi_vivos_g]),hist_psi_vivos_r])
                hist_psi_mortos = np.concatenate([np.concatenate([hist_psi_mortos_b,hist_psi_mortos_g]),hist_psi_mortos_r])
                    
                index = os.path.splitext(os.path.basename(imagem_path))[0]
                file_path = os.path.join(f"{histograms_dir}/{classe}", f"{index}_phi_vivos_bgr.pkl")
                joblib.dump(hist_phi_vivos, file_path)
                file_path = os.path.join(f"{histograms_dir}/{classe}", f"{index}_phi_mortos_bgr.pkl")
                joblib.dump(hist_phi_mortos, file_path)
                file_path = os.path.join(f"{histograms_dir}/{classe}", f"{index}_psi_vivos_bgr.pkl")
                joblib.dump(hist_psi_vivos, file_path)
                file_path = os.path.join(f"{histograms_dir}/{classe}", f"{index}_psi_mortos_bgr.pkl")
                joblib.dump(hist_psi_mortos, file_path)
            except:
                print("Erro, analise a pasta de arquivos")



    print("Progresso concluído")

In [8]:
gerador_histogramas_RGB(dataset_dir)

beaches
bus
dinosaurs
elephants
flowers
foods
horses
monuments
mountains_and_snow
peolpe_and_villages_in_Africa
Progresso concluído


### Testando modelos de aprendizado

#### Carregando histogramas

In [10]:
def carrega_hist_rgb(imagem_path):
    #O objetivo dessa função é carregar os histogramas de determinada imagem

    #data\dataset_1\images\classe\imagem.jpg
    imagem_path = imagem_path.split("/")
    classe, imagem = imagem_path[4], imagem_path[5]
    imagem = imagem.split(".")[0]

    
    histogramas_dir = f"{dataset_dir}/histograms_rgb"
    try:
        hist_phi_vivos = joblib.load(f"{histogramas_dir}/{classe}/{imagem}_phi_vivos_bgr.pkl")
        hist_phi_mortos = joblib.load(f"{histogramas_dir}/{classe}/{imagem}_phi_mortos_bgr.pkl")
        hist_psi_vivos = joblib.load(f"{histogramas_dir}/{classe}/{imagem}_psi_vivos_bgr.pkl")
        hist_psi_mortos = joblib.load(f"{histogramas_dir}/{classe}/{imagem}_psi_mortos_bgr.pkl")

        return hist_phi_vivos, hist_phi_mortos, hist_psi_vivos, hist_psi_mortos

    except:
        print("Os histogramas não foram encontrados. Verifique os diretórios e se os histogramas foram criados corretamente.")

In [11]:
def retorna_combinacao_rgb(imagem_path, combinacao):
    # O objetivo dessa funcao é criar combinacoes com os histogramas da imagem

    hist_phi_vivos, hist_phi_mortos, hist_psi_vivos, hist_psi_mortos = carrega_hist_rgb(imagem_path)
    
    switch = {
        0: hist_phi_vivos,
        1: hist_phi_mortos,
        2: hist_psi_vivos,
        3: hist_psi_mortos,
        4: np.concatenate([hist_phi_vivos, hist_phi_mortos]),   #histogramas phi
        5: np.concatenate([hist_psi_vivos, hist_psi_mortos]),   #histogramas psi
        6: np.concatenate([hist_phi_vivos, hist_psi_vivos]),    #histogramas vivos
        7: np.concatenate([hist_phi_mortos, hist_psi_mortos]),  #histogramas mortos
        8: np.concatenate([np.concatenate([hist_phi_vivos, hist_phi_mortos]),
                           np.concatenate([hist_psi_vivos, hist_psi_mortos])])
    }
    
    return switch.get(combinacao, None)

#### Preparando os dados

In [13]:
# Prepara os dados para treinar o modelo

histogramas_rgb = []
rotulos_rgb = []

for classe in os.listdir(images_dir):
    dir_classe = f"{images_dir}/{classe}"
    print(classe)
    for imagem in os.listdir(dir_classe):
        imagem_path = f"{images_dir}/{classe}/{imagem}"

        #Pode alterar a combinacao 
        combinacao = retorna_combinacao_rgb(imagem_path, 8)

        #Lista dos histogramas
        histogramas_rgb.append(combinacao)
        #lista da classe assossiada ao histograma
        rotulos_rgb.append(classe)

print("Progresso concluído")

beaches
bus
dinosaurs
elephants
flowers
foods
horses
monuments
mountains_and_snow
peolpe_and_villages_in_Africa
Progresso concluído


In [14]:
X = np.array(histogramas_rgb)
y = np.array(rotulos_rgb)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

In [15]:
#Utilizando tecnicas de normalizacao
# 1 = MinMaxScaler, 2 = StandardScaler, 3 = MaxAbsScaler, 4 = RobustScaler
selectedNormalization = 1

if selectedNormalization == 1:
    scaler = preprocessing.MinMaxScaler()
if selectedNormalization == 2:
    scaler = preprocessing.StandardScaler()
if selectedNormalization == 3:
    scaler = preprocessing.MaxAbsScaler()
if selectedNormalization == 4:
    scaler = preprocessing.RobustScaler()

# Escalando os dados de treinamento
X_train = scaler.fit_transform(X_train)
# Escalando os dados de teste com os dados de treinamento, visto que os dados de teste podem ser apenas 1 amostra
X_test = scaler.transform(X_test)

#### Testando modelos

In [16]:
# Gaussian Naive Bayes
gnb = GaussianNB()
model1 = gnb.fit(X_train, y_train)
aux = gnb.predict(X_test)
print(classification_report(y_test, aux, zero_division=1))
#print(confusion_matrix(y_test, aux))

                               precision    recall  f1-score   support

                      beaches       0.59      0.50      0.54        20
                          bus       0.42      0.50      0.45        20
                    dinosaurs       1.00      0.05      0.10        20
                    elephants       0.39      0.80      0.52        20
                      flowers       0.68      0.85      0.76        20
                        foods       0.33      0.50      0.40        20
                       horses       0.80      0.40      0.53        20
                    monuments       0.38      0.30      0.33        20
           mountains_and_snow       0.63      0.60      0.62        20
peolpe_and_villages_in_Africa       0.65      0.55      0.59        20

                     accuracy                           0.51       200
                    macro avg       0.59      0.51      0.48       200
                 weighted avg       0.59      0.51      0.48       200



In [27]:
# Logistic Regression
logreg = LogisticRegression(max_iter=1000)
model2 = logreg.fit(X_train, y_train)
aux = logreg.predict(X_test)
print(classification_report(y_test, aux, zero_division=1))
#print(confusion_matrix(y_test, aux))

                               precision    recall  f1-score   support

                      beaches       0.58      0.35      0.44        20
                          bus       0.88      0.75      0.81        20
                    dinosaurs       0.95      1.00      0.98        20
                    elephants       0.77      0.85      0.81        20
                      flowers       0.80      1.00      0.89        20
                        foods       0.75      0.75      0.75        20
                       horses       0.94      0.85      0.89        20
                    monuments       0.72      0.65      0.68        20
           mountains_and_snow       0.64      0.80      0.71        20
peolpe_and_villages_in_Africa       0.68      0.75      0.71        20

                     accuracy                           0.78       200
                    macro avg       0.77      0.78      0.77       200
                 weighted avg       0.77      0.78      0.77       200



In [18]:
# Decision Tree
dectree = DecisionTreeClassifier()
model3 = dectree.fit(X_train, y_train)
aux = dectree.predict(X_test)
print(classification_report(y_test, aux, zero_division=1))
#print(confusion_matrix(y_test, aux))

                               precision    recall  f1-score   support

                      beaches       0.70      0.35      0.47        20
                          bus       0.75      0.75      0.75        20
                    dinosaurs       0.95      0.95      0.95        20
                    elephants       0.65      0.65      0.65        20
                      flowers       0.90      0.95      0.93        20
                        foods       0.44      0.60      0.51        20
                       horses       0.76      0.80      0.78        20
                    monuments       0.37      0.50      0.43        20
           mountains_and_snow       0.37      0.35      0.36        20
peolpe_and_villages_in_Africa       0.67      0.50      0.57        20

                     accuracy                           0.64       200
                    macro avg       0.66      0.64      0.64       200
                 weighted avg       0.66      0.64      0.64       200



In [19]:
# K-Nearest Neighbors
knn = KNeighborsClassifier(n_neighbors = 3)
model4 = knn.fit(X_train, y_train)
aux = knn.predict(X_test)
print(classification_report(y_test, aux, zero_division=1))
#print(confusion_matrix(y_test, aux))

                               precision    recall  f1-score   support

                      beaches       0.56      0.45      0.50        20
                          bus       0.55      0.30      0.39        20
                    dinosaurs       1.00      1.00      1.00        20
                    elephants       0.52      0.60      0.56        20
                      flowers       0.75      0.90      0.82        20
                        foods       0.45      0.65      0.53        20
                       horses       0.95      0.95      0.95        20
                    monuments       0.35      0.30      0.32        20
           mountains_and_snow       0.50      0.15      0.23        20
peolpe_and_villages_in_Africa       0.44      0.75      0.56        20

                     accuracy                           0.60       200
                    macro avg       0.61      0.61      0.59       200
                 weighted avg       0.61      0.60      0.59       200



In [20]:
# Linear Discriminant Analysis
lda = LinearDiscriminantAnalysis()
model5 = lda.fit(X_train, y_train)
aux = lda.predict(X_test)
print(classification_report(y_test, aux, zero_division=1))
#print(confusion_matrix(y_test, aux))

                               precision    recall  f1-score   support

                      beaches       0.45      0.25      0.32        20
                          bus       0.78      0.70      0.74        20
                    dinosaurs       1.00      1.00      1.00        20
                    elephants       0.68      0.85      0.76        20
                      flowers       0.78      0.90      0.84        20
                        foods       0.56      0.50      0.53        20
                       horses       0.80      0.80      0.80        20
                    monuments       0.67      0.50      0.57        20
           mountains_and_snow       0.55      0.55      0.55        20
peolpe_and_villages_in_Africa       0.57      0.85      0.68        20

                     accuracy                           0.69       200
                    macro avg       0.68      0.69      0.68       200
                 weighted avg       0.68      0.69      0.68       200



In [21]:
# Support Vector Machine
svm = SVC()
model6 = svm.fit(X_train, y_train)
aux = svm.predict(X_test)
print(classification_report(y_test, aux, zero_division=1))
#print(confusion_matrix(y_test, aux))

                               precision    recall  f1-score   support

                      beaches       0.53      0.45      0.49        20
                          bus       0.76      0.80      0.78        20
                    dinosaurs       0.95      1.00      0.98        20
                    elephants       0.81      0.85      0.83        20
                      flowers       0.87      1.00      0.93        20
                        foods       0.70      0.70      0.70        20
                       horses       1.00      0.95      0.97        20
                    monuments       0.54      0.35      0.42        20
           mountains_and_snow       0.57      0.65      0.60        20
peolpe_and_villages_in_Africa       0.64      0.70      0.67        20

                     accuracy                           0.74       200
                    macro avg       0.74      0.74      0.74       200
                 weighted avg       0.74      0.74      0.74       200



In [22]:
# RandomForest
rf = RandomForestClassifier()
model7 = rf.fit(X_train, y_train)
aux = rf.predict(X_test)
print(classification_report(y_test, aux, zero_division=1))
#print(confusion_matrix(y_test, aux))

                               precision    recall  f1-score   support

                      beaches       0.81      0.65      0.72        20
                          bus       0.75      0.75      0.75        20
                    dinosaurs       0.87      1.00      0.93        20
                    elephants       0.81      0.85      0.83        20
                      flowers       0.90      0.90      0.90        20
                        foods       0.67      0.80      0.73        20
                       horses       0.90      0.95      0.93        20
                    monuments       0.76      0.65      0.70        20
           mountains_and_snow       0.67      0.60      0.63        20
peolpe_and_villages_in_Africa       0.75      0.75      0.75        20

                     accuracy                           0.79       200
                    macro avg       0.79      0.79      0.79       200
                 weighted avg       0.79      0.79      0.79       200



In [23]:
# Neural Net
nnet = MLPClassifier(alpha=1)
model8 = nnet.fit(X_train, y_train)
aux = nnet.predict(X_test)
print(classification_report(y_test, aux, zero_division=1))
#print(confusion_matrix(y_test, aux))

                               precision    recall  f1-score   support

                      beaches       0.67      0.40      0.50        20
                          bus       0.88      0.70      0.78        20
                    dinosaurs       0.95      1.00      0.98        20
                    elephants       0.83      0.75      0.79        20
                      flowers       0.83      1.00      0.91        20
                        foods       0.81      0.85      0.83        20
                       horses       0.95      0.90      0.92        20
                    monuments       0.71      0.75      0.73        20
           mountains_and_snow       0.67      0.80      0.73        20
peolpe_and_villages_in_Africa       0.62      0.75      0.68        20

                     accuracy                           0.79       200
                    macro avg       0.79      0.79      0.78       200
                 weighted avg       0.79      0.79      0.78       200



# Caltech256

## Definindo diretórios

In [34]:
data_dir = f"./data"

#caltech256
dataset_1_dir = f"{data_dir}/dataset_1"
images_1_dir = f"{dataset_1_dir}/images"

## Em RBG:


### Gerando histogramas

In [ ]:
def obter_vizinhos_rgb(matriz_de_intensidade, linha, coluna):
    # O objetivo dessa função é retornar uma lista contendo a intensidade dos 9 vizinhos do pixel observado

    #Esse método de partição da matriz foi escolhido para manter a analise dos pixels vizinhos
    #dentro dos limites da matriz de intensidade, evitando assim valores negativos ou fora do shape.
    vizinhos = matriz_de_intensidade[max(0, linha-1):min(matriz_de_intensidade.shape[0], linha+2),
                             max(0, coluna-1):min(matriz_de_intensidade.shape[1], coluna+2)]

    #Transforma a matriz particionada em lista e remove o pixel do centro
    lista_de_vizinhos = vizinhos.flatten().tolist()
    lista_de_vizinhos.remove(matriz_de_intensidade[linha, coluna])

    return lista_de_vizinhos

In [ ]:
def aplica_regras_rgb(lista_de_vizinhos, intensidade_pixel_central, estado_pixel_central):
    #O objetivo dessa função é aplicar as regras do jogo da vida de Conway no pixel observado e retornar seu estado.
    # 1 = vivo, 0 = morto

    #Conta o número de vizinhos vivos com a mesma intensidade que o pixel central
    vizinhos_iguais = lista_de_vizinhos.count(intensidade_pixel_central)

    #Regra 1: A célula viva com dois ou três vizinhos vivos sobrevive
    if estado_pixel_central == 1 and (vizinhos_iguais == 2 or vizinhos_iguais == 3):
        return 1

    #Regra 2: A célula viva com menos de dois vizinhos vivos morre (subpopulação)
    elif estado_pixel_central == 1 and vizinhos_iguais < 2:
        return 0

    #Regra 3: A célula viva com mais de três vizinhos vivos morre (superpopulação)
    elif estado_pixel_central == 1 and vizinhos_iguais > 3:
        return 0

    #Regra 4: A célula morta com exatamente três vizinhos vivos se torna viva (resurreição)
    elif estado_pixel_central == 0 and vizinhos_iguais == 3:
        return 1

    #Pra todos os outros casos, a célula permanece no mesmo estado
    else:
        return estado_pixel_central

In [ ]:
def percorre_imagem_aplicando_regras_rgb(matriz_de_estados, matriz_de_intensidade):
    # O objetivo dessa função é percorrer a imagem, chamar a função para obter os vizinhos e aplicar as regras

    linhas, colunas = matriz_de_intensidade.shape
    for linha in range(linhas):
        for coluna in range(colunas):
            #Obtem os vizinhos do pixel atual
            lista_de_vizinhos = obter_vizinhos_rgb(matriz_de_intensidade, linha, coluna)
            #Aplica as regras do jogo da vida no pixel atual (atualiza a matriz de estado inicial)
            matriz_de_estados[linha, coluna] = aplica_regras_rgb(lista_de_vizinhos, matriz_de_intensidade[linha, coluna], matriz_de_estados[linha, coluna])

    return matriz_de_estados

In [ ]:
def gera_histogramas_rgb(imagem):
    #O objetivo dessa função é criar as matriz de intensidade e as de estado inicial para cada imagem
    #Após aplicar as regras cria os histogramas

    #Transforma a imagem em uma matriz de intensidade
    matriz_de_intensidade = np.array(imagem)

    #Cria as matrizes de estados iniciais
    matriz_de_estados_phi = np.ones(matriz_de_intensidade.shape, dtype=int) #todos vivos
    matriz_de_estados_psi = np.zeros(matriz_de_intensidade.shape, dtype=int) #todos mortos

    #Aplica as regras do jogo da vida e atualiza as matrizes de estado inicial
    matriz_de_estados_phi = percorre_imagem_aplicando_regras_rgb(matriz_de_estados_phi, matriz_de_intensidade)
    matriz_de_estados_psi = percorre_imagem_aplicando_regras_rgb(matriz_de_estados_psi, matriz_de_intensidade)

    #As matrizes são convertidas em listas
    #Phi -> estado inicial = vivo
    phi_vivos = matriz_de_intensidade.flatten()[matriz_de_estados_phi.flatten() == 1] #se manteram vivos
    phi_mortos = matriz_de_intensidade.flatten()[matriz_de_estados_phi.flatten() == 0] #morreram

    #Psi -> estado inicial = morto
    psi_vivos = matriz_de_intensidade.flatten()[matriz_de_estados_psi.flatten() == 1] #ressuscitaram
    psi_mortos = matriz_de_intensidade.flatten()[matriz_de_estados_psi.flatten() == 0] #se manteram mortos

    #Cria os histogramas
    hist_phi_vivos, _ = np.histogram(phi_vivos, bins=256, range=(0, 256))
    hist_phi_mortos, _ = np.histogram(phi_mortos, bins=256, range=(0, 256))
    hist_psi_vivos, _ = np.histogram(psi_vivos, bins=256, range=(0, 256))
    hist_psi_mortos, _ = np.histogram(psi_mortos, bins=256, range=(0, 256))

    return hist_phi_vivos, hist_phi_mortos, hist_psi_vivos, hist_psi_mortos


In [ ]:
from multiprocessing import Pool

def gerador_histogramas_RGB(dataset_dir):
    images_dir = f"{dataset_dir}/images"
    histograms_dir = f"{dataset_dir}/histograms_rgb"
    os.makedirs(histograms_dir, exist_ok=True)

    classes = [d for d in os.listdir(images_dir) if os.path.isdir(os.path.join(images_dir, d))]
    for classe in classes:
        os.makedirs(os.path.join(histograms_dir, classe), exist_ok=True)

    i = 1
    for classe in classes:
        print(classe)
        dir_classe = os.path.join(images_dir, classe)
        for imagem in os.listdir(dir_classe):
            try:
                imagem_path = os.path.join(dir_classe, imagem)
                imagem_rgb = cv2.imread(imagem_path, cv2.IMREAD_COLOR)
                imagem_rgb = cv2.resize(imagem_rgb, (128, 128))  # Resize para 128x128
                
                hist_phi_vivos_b, hist_phi_mortos_b, hist_psi_vivos_b, hist_psi_mortos_b = gera_histogramas_rgb(imagem_rgb[:, :, 0])
                hist_phi_vivos_g, hist_phi_mortos_g, hist_psi_vivos_g, hist_psi_mortos_g = gera_histogramas_rgb(imagem_rgb[:, :, 1])
                hist_phi_vivos_r, hist_phi_mortos_r, hist_psi_vivos_r, hist_psi_mortos_r = gera_histogramas_rgb(imagem_rgb[:, :, 2])
                
                hist_phi_vivos = np.concatenate([np.concatenate([hist_phi_vivos_b,hist_phi_vivos_g]),hist_phi_vivos_r])
                hist_phi_mortos = np.concatenate([np.concatenate([hist_phi_mortos_b,hist_phi_mortos_g]),hist_phi_mortos_r])
                hist_psi_vivos = np.concatenate([np.concatenate([hist_psi_vivos_b,hist_psi_vivos_g]),hist_psi_vivos_r])
                hist_psi_mortos = np.concatenate([np.concatenate([hist_psi_mortos_b,hist_psi_mortos_g]),hist_psi_mortos_r])
                    
                index = os.path.splitext(os.path.basename(imagem_path))[0]
                file_path = os.path.join(f"{histograms_dir}/{classe}", f"{index}_phi_vivos_bgr.pkl")
                joblib.dump(hist_phi_vivos, file_path)
                file_path = os.path.join(f"{histograms_dir}/{classe}", f"{index}_phi_mortos_bgr.pkl")
                joblib.dump(hist_phi_mortos, file_path)
                file_path = os.path.join(f"{histograms_dir}/{classe}", f"{index}_psi_vivos_bgr.pkl")
                joblib.dump(hist_psi_vivos, file_path)
                file_path = os.path.join(f"{histograms_dir}/{classe}", f"{index}_psi_mortos_bgr.pkl")
                joblib.dump(hist_psi_mortos, file_path)
            except:
                print("Erro, analise a pasta de arquivos")



    print("Progresso concluído")

In [ ]:
#gerador_histogramas_RGB(dataset_1_dir)

### Testando modelos de aprendizado 

In [ ]:
def carrega_hist_rgb(imagem_path):
    #O objetivo dessa função é carregar os histogramas de determinada imagem

    #data\dataset_1\images\classe\imagem.jpg
    imagem_path = imagem_path.split("/")
    classe, imagem = imagem_path[4], imagem_path[5]
    imagem = imagem.split(".")[0]

    
    histogramas_dir = f"{dataset_1_dir}/histograms_rgb"
    try:
        hist_phi_vivos = joblib.load(f"{histogramas_dir}/{classe}/{imagem}_phi_vivos_bgr.pkl")
        hist_phi_mortos = joblib.load(f"{histogramas_dir}/{classe}/{imagem}_phi_mortos_bgr.pkl")
        hist_psi_vivos = joblib.load(f"{histogramas_dir}/{classe}/{imagem}_psi_vivos_bgr.pkl")
        hist_psi_mortos = joblib.load(f"{histogramas_dir}/{classe}/{imagem}_psi_mortos_bgr.pkl")

        return hist_phi_vivos, hist_phi_mortos, hist_psi_vivos, hist_psi_mortos

    except:
        print("Os histogramas não foram encontrados. Verifique os diretórios e se os histogramas foram criados corretamente.")

In [ ]:
def retorna_combinacao_rgb(imagem_path, combinacao):
    # O objetivo dessa funcao é criar combinacoes com os histogramas da imagem

    hist_phi_vivos, hist_phi_mortos, hist_psi_vivos, hist_psi_mortos = carrega_hist_rgb(imagem_path)
    
    switch = {
        0: hist_phi_vivos,
        1: hist_phi_mortos,
        2: hist_psi_vivos,
        3: hist_psi_mortos,
        4: np.concatenate([hist_phi_vivos, hist_phi_mortos]),   #histogramas phi
        5: np.concatenate([hist_psi_vivos, hist_psi_mortos]),   #histogramas psi
        6: np.concatenate([hist_phi_vivos, hist_psi_vivos]),    #histogramas vivos
        7: np.concatenate([hist_phi_mortos, hist_psi_mortos]),  #histogramas mortos
        8: np.concatenate([np.concatenate([hist_phi_vivos, hist_phi_mortos]),
                           np.concatenate([hist_psi_vivos, hist_psi_mortos])])
    }
    
    return switch.get(combinacao, None)

In [ ]:
## 0 -> "hist_phi_vivos",       1 -> "hist_phi_mortos",   2 -> "hist_psi_vivos",         3 -> "hist_psi_mortos",
## 4 -> "hist_phi_combined",    5 -> "hist_psi_combined", 6 -> "hist_vivos_combined",
## 7 -> "hist_mortos_combined", 8 -> "all_combined" 

In [ ]:
# Prepara os dados para treinar o modelo

histogramas_rgb = []
rotulos_rgb = []

for classe in os.listdir(images_1_dir):
    dir_classe = f"{images_1_dir}/{classe}"
    #print(classe)
    for imagem in os.listdir(dir_classe):
        imagem_path = f"{images_1_dir}/{classe}/{imagem}"

        #Pode alterar a combinacao 
        combinacao = retorna_combinacao_rgb(imagem_path, 8)

        #Lista dos histogramas
        histogramas_rgb.append(combinacao)
        #lista da classe assossiada ao histograma
        rotulos_rgb.append(classe)

print("Progresso concluído")

In [ ]:
X = np.array(histogramas_rgb)
y = np.array(rotulos_rgb)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

In [ ]:
#Utilizando tecnicas de normalizacao
# 1 = MinMaxScaler, 2 = StandardScaler, 3 = MaxAbsScaler, 4 = RobustScaler
selectedNormalization = 1

if selectedNormalization == 1:
    scaler = preprocessing.MinMaxScaler()
if selectedNormalization == 2:
    scaler = preprocessing.StandardScaler()
if selectedNormalization == 3:
    scaler = preprocessing.MaxAbsScaler()
if selectedNormalization == 4:
    scaler = preprocessing.RobustScaler()

# Escalando os dados de treinamento
X_train = scaler.fit_transform(X_train)
# Escalando os dados de teste com os dados de treinamento, visto que os dados de teste podem ser apenas 1 amostra
X_test = scaler.transform(X_test)

In [ ]:
# Gaussian Naive Bayes
gnb = GaussianNB()
model1 = gnb.fit(X_train, y_train)
aux = gnb.predict(X_test)
print(classification_report(y_test, aux, zero_division=1))
#print(confusion_matrix(y_test, aux))

# acuracia sem scaler   == 0.05
# acuriacia com scaler1 == 0.05

In [ ]:
# Logistic Regression
logreg = LogisticRegression()
model2 = logreg.fit(X_train, y_train)
aux = logreg.predict(X_test)
print(classification_report(y_test, aux, zero_division=1))
#print(confusion_matrix(y_test, aux))

# acuracia sem scaler == 0.11
# acuriacia com scaler1 == 0.10

In [ ]:
# Decision Tree
dectree = DecisionTreeClassifier()
model3 = dectree.fit(X_train, y_train)
aux = dectree.predict(X_test)
print(classification_report(y_test, aux, zero_division=1))
#print(confusion_matrix(y_test, aux))

# acuracia sem scaler   == 0.07
# acuriacia com scaler1 == 0.07

In [ ]:
# K-Nearest Neighbors
knn = KNeighborsClassifier(n_neighbors = 3)
model4 = knn.fit(X_train, y_train)
aux = knn.predict(X_test)
print(classification_report(y_test, aux, zero_division=1))
#print(confusion_matrix(y_test, aux))

# acuracia sem scaler   == 0.07
# acuriacia com scaler1 == 0.07

In [ ]:
# Linear Discriminant Analysis
lda = LinearDiscriminantAnalysis()
model5 = lda.fit(X_train, y_train)
aux = lda.predict(X_test)
print(classification_report(y_test, aux, zero_division=1))
#print(confusion_matrix(y_test, aux))

# acuracia sem scaler   == 0.08
# acuriacia com scaler1 == 0.08

In [ ]:
# Support Vector Machine
svm = SVC()
model6 = svm.fit(X_train, y_train)
aux = svm.predict(X_test)
print(classification_report(y_test, aux, zero_division=1))
#print(confusion_matrix(y_test, aux))

# acuracia sem scaler   == 0.10
# acuriacia com scaler1 == 0.12

In [ ]:
# RandomForest
rf = RandomForestClassifier()
model7 = rf.fit(X_train, y_train)
aux = rf.predict(X_test)
print(classification_report(y_test, aux, zero_division=1))
#print(confusion_matrix(y_test, aux))

# acuracia sem scaler   == 0.15
# acuriacia com scaler1 == 0.15

In [ ]:
# Neural Net
nnet = MLPClassifier(alpha=1)
model8 = nnet.fit(X_train, y_train)
aux = nnet.predict(X_test)
print(classification_report(y_test, aux, zero_division=1))
#print(confusion_matrix(y_test, aux))

# acuracia sem scaler   == 0.03
# acuriacia com scaler1 == 0.07

## Em escala de cinza:

### Gerando histogramas

In [ ]:
def obter_vizinhos(matriz_de_intensidade, linha, coluna):
    # O objetivo dessa função é retornar uma lista contendo a intensidade dos 9 vizinhos do pixel observado

    #Esse método de partição da matriz foi escolhido para manter a analise dos pixels vizinhos
    #dentro dos limites da matriz de intensidade, evitando assim valores negativos ou fora do shape.
    vizinhos = matriz_de_intensidade[max(0, linha-1):min(matriz_de_intensidade.shape[0], linha+2),
                             max(0, coluna-1):min(matriz_de_intensidade.shape[1], coluna+2)]

    #Transforma a matriz particionada em lista e remove o pixel do centro
    lista_de_vizinhos = vizinhos.flatten().tolist()
    lista_de_vizinhos.remove(matriz_de_intensidade[linha, coluna])

    return lista_de_vizinhos

In [ ]:
def aplica_regras(lista_de_vizinhos, intensidade_pixel_central, estado_pixel_central):
    #O objetivo dessa função é aplicar as regras do jogo da vida de Conway no pixel observado e retornar seu estado.
    # 1 = vivo, 0 = morto

    #Conta o número de vizinhos vivos com a mesma intensidade que o pixel central
    vizinhos_iguais = lista_de_vizinhos.count(intensidade_pixel_central)

    #Regra 1: A célula viva com dois ou três vizinhos vivos sobrevive
    if estado_pixel_central == 1 and (vizinhos_iguais == 2 or vizinhos_iguais == 3):
        return 1

    #Regra 2: A célula viva com menos de dois vizinhos vivos morre (subpopulação)
    elif estado_pixel_central == 1 and vizinhos_iguais < 2:
        return 0

    #Regra 3: A célula viva com mais de três vizinhos vivos morre (superpopulação)
    elif estado_pixel_central == 1 and vizinhos_iguais > 3:
        return 0

    #Regra 4: A célula morta com exatamente três vizinhos vivos se torna viva (resurreição)
    elif estado_pixel_central == 0 and vizinhos_iguais == 3:
        return 1

    #Pra todos os outros casos, a célula permanece no mesmo estado
    else:
        return estado_pixel_central

In [ ]:
def percorre_imagem_aplicando_regras(matriz_de_estados, matriz_de_intensidade):
    # O objetivo dessa função é percorrer a imagem, chamar a função para obter os vizinhos e aplicar as regras

    linhas, colunas = matriz_de_intensidade.shape
    for linha in range(linhas):
        for coluna in range(colunas):
            #Obtem os vizinhos do pixel atual
            lista_de_vizinhos = obter_vizinhos(matriz_de_intensidade, linha, coluna)
            #Aplica as regras do jogo da vida no pixel atual (atualiza a matriz de estado inicial)
            matriz_de_estados[linha, coluna] = aplica_regras(lista_de_vizinhos, matriz_de_intensidade[linha, coluna], matriz_de_estados[linha, coluna])

    return matriz_de_estados

In [ ]:
def gera_histogramas(imagem_cinza):
    #O objetivo dessa função é criar as matriz de intensidade e as de estado inicial para cada imagem
    #Após aplicar as regras cria os histogramas

    #Transforma a imagem em uma matriz de intensidade
    matriz_de_intensidade = np.array(imagem_cinza)

    #Cria as matrizes de estados iniciais
    matriz_de_estados_phi = np.ones(matriz_de_intensidade.shape, dtype=int) #todos vivos
    matriz_de_estados_psi = np.zeros(matriz_de_intensidade.shape, dtype=int) #todos mortos

    #Aplica as regras do jogo da vida e atualiza as matrizes de estado inicial
    matriz_de_estados_phi = percorre_imagem_aplicando_regras(matriz_de_estados_phi, matriz_de_intensidade)
    matriz_de_estados_psi = percorre_imagem_aplicando_regras(matriz_de_estados_psi, matriz_de_intensidade)

    #As matrizes são convertidas em listas
    #Phi -> estado inicial = vivo
    phi_vivos = matriz_de_intensidade.flatten()[matriz_de_estados_phi.flatten() == 1] #se manteram vivos
    phi_mortos = matriz_de_intensidade.flatten()[matriz_de_estados_phi.flatten() == 0] #morreram

    #Psi -> estado inicial = morto
    psi_vivos = matriz_de_intensidade.flatten()[matriz_de_estados_psi.flatten() == 1] #ressuscitaram
    psi_mortos = matriz_de_intensidade.flatten()[matriz_de_estados_psi.flatten() == 0] #se manteram mortos

    #Cria os histogramas
    hist_phi_vivos, _ = np.histogram(phi_vivos, bins=256, range=(0, 256))
    hist_phi_mortos, _ = np.histogram(phi_mortos, bins=256, range=(0, 256))
    hist_psi_vivos, _ = np.histogram(psi_vivos, bins=256, range=(0, 256))
    hist_psi_mortos, _ = np.histogram(psi_mortos, bins=256, range=(0, 256))

    return hist_phi_vivos, hist_phi_mortos, hist_psi_vivos, hist_psi_mortos

In [ ]:
def gerador_histogramas(dataset_dir):
    #Funcao principal: faz a chamada das funções acima e salva os histogramas

    images_dir = f"{dataset_dir}/images"
    histograms_dir = f"{dataset_dir}/histograms_grayscale"

    # Cria as pastas para salvar os histogramas mantendo o padrão das classes do dataset
    os.makedirs(histograms_dir, exist_ok=True)

    classes = [conteudo_item for conteudo_item in os.listdir(images_dir) if os.path.isdir(os.path.join(images_dir, conteudo_item))]
    for classe in classes:
        dir_pastas = os.path.join(histograms_dir, classe)
        os.makedirs(dir_pastas, exist_ok=True)

    i = 0
    #Esse loop pega a imagem, gera seus histogramas e salva com base no nome da classe e da imagem
    for classe in os.listdir(images_dir):
        dir_classe = f"{images_dir}/{classe}"
        
        i += 1
        if i <= 175:
            pass
        else:
            print(classe)
            for imagem in os.listdir(dir_classe):
                imagem_path = f"{images_dir}/{classe}/{imagem}"
            
                try:
                    imagem_cinza = cv2.imread(imagem_path, cv2.IMREAD_GRAYSCALE)
        
                    imagem_cinza = cv2.resize(imagem_cinza, (128,128))
                    #Dimensao original = 384 * 256 = 98304
                    #Resize = 128 * 128 = 16384
                    #Redução = (16384-98304)/98304*100 = 83,33%
        
                    hist_phi_vivos, hist_phi_mortos, hist_psi_vivos, hist_psi_mortos = gera_histogramas(imagem_cinza)
        
                    index = imagem.split(".")[0]
                    file_path = os.path.join(f"{histograms_dir}/{classe}", f"{index}_phi_vivos.pkl")
                    joblib.dump(hist_phi_vivos, file_path)
                    file_path = os.path.join(f"{histograms_dir}/{classe}", f"{index}_phi_mortos.pkl")
                    joblib.dump(hist_phi_mortos, file_path)
                    file_path = os.path.join(f"{histograms_dir}/{classe}", f"{index}_psi_vivos.pkl")
                    joblib.dump(hist_psi_vivos, file_path)
                    file_path = os.path.join(f"{histograms_dir}/{classe}", f"{index}_psi_mortos.pkl")
                    joblib.dump(hist_psi_mortos, file_path)
                except:
                    print("Um erro foi encontrado, revise o dataset")

    print("Progresso concluído")

In [ ]:
gerador_histogramas(dataset_1_dir)

### Testando modelos de aprendizado

In [ ]:
def carrega_hist(imagem_path):
    #O objetivo dessa função é carregar os histogramas de determinada imagem

    #data\dataset\images\beaches\100.jpg
    imagem_path = imagem_path.split("/")
    classe, imagem = imagem_path[4], imagem_path[5]
    imagem = imagem.split(".")[0]

    i = 0
    histogramas_dir = f"{dataset_1_dir}/histograms_grayscale"
    try:
        i+=1
        hist_phi_vivos = joblib.load(f"{histogramas_dir}/{classe}/{imagem}_phi_vivos.pkl")
        hist_phi_mortos = joblib.load(f"{histogramas_dir}/{classe}/{imagem}_phi_mortos.pkl")
        hist_psi_vivos = joblib.load(f"{histogramas_dir}/{classe}/{imagem}_psi_vivos.pkl")
        hist_psi_mortos = joblib.load(f"{histogramas_dir}/{classe}/{imagem}_psi_mortos.pkl")

        return hist_phi_vivos, hist_phi_mortos, hist_psi_vivos, hist_psi_mortos

    except:
        print("Os histogramas não foram encontrados. Verifique os diretórios e se os histogramas foram criados corretamente.")
    print(i)

In [ ]:
def retorna_combinacao(imagem_path, combinacao):
    # O objetivo dessa funcao é criar combinacoes com os histogramas da imagem

    hist_phi_vivos, hist_phi_mortos, hist_psi_vivos, hist_psi_mortos = carrega_hist(imagem_path)
    if combinacao == 0:
        return hist_phi_vivos
    elif combinacao == 1:
        return hist_phi_mortos
    elif combinacao == 2:
        return hist_psi_vivos
    elif combinacao == 3:
        return hist_psi_mortos
    elif combinacao == 4:
        return np.concatenate([hist_phi_vivos, hist_phi_mortos]) #histogramas phi
    elif combinacao == 5:
        return np.concatenate([hist_psi_vivos, hist_psi_mortos]) #histogramas psi
    elif combinacao == 6:
        return np.concatenate([hist_phi_vivos, hist_psi_vivos]) #histogramas vivos
    elif combinacao == 7:
        return np.concatenate([hist_phi_mortos, hist_psi_mortos]) #histogramas mortos
    elif combinacao == 8:
        return np.concatenate([np.concatenate([hist_phi_vivos, hist_phi_mortos]),
                               np.concatenate([hist_psi_vivos, hist_psi_mortos])])
        #histogramas phi e psi combinados

In [ ]:
# Prepara os dados para treinar o modelo

histogramas = []
rotulos = []

for classe in os.listdir(images_1_dir):
    dir_classe = f"{images_1_dir}/{classe}"
    print(classe)
    for imagem in os.listdir(dir_classe):
        imagem_path = f"{images_1_dir}/{classe}/{imagem}"

        #Pode alterar a combinacao
        combinacao = retorna_combinacao(imagem_path, 8)

        #Lista dos histogramas
        histogramas.append(combinacao)
        #lista da classe assossiada ao histograma
        rotulos.append(classe)

print("Progresso concluído")

In [ ]:
X = np.array(histogramas)
y = np.array(rotulos)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1,stratify=y)

In [ ]:
#Utilizando tecnicas de normalizacao
# 1 = MinMaxScaler, 2 = StandardScaler, 3 = MaxAbsScaler, 4 = RobustScaler
selectedNormalization = 1

if selectedNormalization == 1:
    scaler = preprocessing.MinMaxScaler()
if selectedNormalization == 2:
    scaler = preprocessing.StandardScaler()
if selectedNormalization == 3:
    scaler = preprocessing.MaxAbsScaler()
if selectedNormalization == 4:
    scaler = preprocessing.RobustScaler()

# Escalando os dados de treinamento
X_train = scaler.fit_transform(X_train)
# Escalando os dados de teste com os dados de treinamento, visto que os dados de teste podem ser apenas 1 amostra
X_test = scaler.transform(X_test)

In [ ]:
# Gaussian Naive Bayes
gnb = GaussianNB()
model1 = gnb.fit(X_train, y_train)
aux = gnb.predict(X_test)
print(classification_report(y_test, aux, zero_division=1))
#print(confusion_matrix(y_test, aux))

In [ ]:
# Logistic Regression
logreg = LogisticRegression()
model2 = logreg.fit(X_train, y_train)
aux = logreg.predict(X_test)
print(classification_report(y_test, aux, zero_division=1))
#print(confusion_matrix(y_test, aux))

In [ ]:
# Decision Tree
dectree = DecisionTreeClassifier()
model3 = dectree.fit(X_train, y_train)
aux = dectree.predict(X_test)
print(classification_report(y_test, aux, zero_division=1))
#print(confusion_matrix(y_test, aux))

In [ ]:
# K-Nearest Neighbors
knn = KNeighborsClassifier(n_neighbors = 3)
model4 = knn.fit(X_train, y_train)
aux = knn.predict(X_test)
print(classification_report(y_test, aux, zero_division=1))
#print(confusion_matrix(y_test, aux))

In [ ]:
# Linear Discriminant Analysis
lda = LinearDiscriminantAnalysis()
model5 = lda.fit(X_train, y_train)
aux = lda.predict(X_test)
print(classification_report(y_test, aux, zero_division=1))
#print(confusion_matrix(y_test, aux))

In [ ]:
# Support Vector Machine
svm = SVC()
model6 = svm.fit(X_train, y_train)
aux = svm.predict(X_test)
print(classification_report(y_test, aux, zero_division=1))
#print(confusion_matrix(y_test, aux))

In [ ]:
# RandomForest
rf = RandomForestClassifier()
model7 = rf.fit(X_train, y_train)
aux = rf.predict(X_test)
print(classification_report(y_test, aux, zero_division=1))
#print(confusion_matrix(y_test, aux))

In [ ]:
# Neural Net
nnet = MLPClassifier(alpha=1)
model8 = nnet.fit(X_train, y_train)
aux = nnet.predict(X_test)
print(classification_report(y_test, aux, zero_division=1))
#print(confusion_matrix(y_test, aux))